# TV Script Generation

In this project, you'll generate your own [Seinfeld](https://en.wikipedia.org/wiki/Seinfeld) TV scripts using RNNs.  You'll be using part of the [Seinfeld dataset](https://www.kaggle.com/thec03u5/seinfeld-chronicles#scripts.csv) of scripts from 9 seasons.  The Neural Network you'll build will generate a new ,"fake" TV script, based on patterns it recognizes in this training data.

## Get the Data

The data is already provided for you in `./data/Seinfeld_Scripts.txt` and you're encouraged to open that file and look at the text. 
>* As a first step, we'll load in this data and look at some samples. 
* Then, you'll be tasked with defining and training an RNN to generate a new script!

In [1]:
"""
DON'T MODIFY ANYTHING IN THIS CELL
"""
# load in data
import helper
data_dir = './data/Seinfeld_Scripts.txt'
text = helper.load_data(data_dir)

## Explore the Data
Play around with `view_line_range` to view different parts of the data. This will give you a sense of the data you'll be working with. You can see, for example, that it is all lowercase text, and each new line of dialogue is separated by a newline character `\n`.

In [2]:
view_line_range = (0, 10)

"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
import numpy as np

print('Dataset Stats')
print('Roughly the number of unique words: {}'.format(len({word: None for word in text.split()})))

lines = text.split('\n')
print('Number of lines: {}'.format(len(lines)))
word_count_line = [len(line.split()) for line in lines]
print('Average number of words in each line: {}'.format(np.average(word_count_line)))

print()
print('The lines {} to {}:'.format(*view_line_range))
print('\n'.join(text.split('\n')[view_line_range[0]:view_line_range[1]]))

Dataset Stats
Roughly the number of unique words: 46367
Number of lines: 109233
Average number of words in each line: 5.544240293684143

The lines 0 to 10:
jerry: do you know what this is all about? do you know, why were here? to be out, this is out...and out is one of the single most enjoyable experiences of life. people...did you ever hear people talking about we should go out? this is what theyre talking about...this whole thing, were all out now, no one is home. not one person here is home, were all out! there are people trying to find us, they dont know where we are. (on an imaginary phone) did you ring?, i cant find him. where did he go? he didnt tell me where he was going. he must have gone out. you wanna go out you get ready, you pick out the clothes, right? you take the shower, you get all ready, get the cash, get your friends, the car, the spot, the reservation...then youre standing around, what do you do? you go we gotta be getting back. once youre out, you wanna get back! y

---
## Implement Pre-processing Functions
The first thing to do to any dataset is pre-processing.  Implement the following pre-processing functions below:
- Lookup Table
- Tokenize Punctuation

### Lookup Table
To create a word embedding, you first need to transform the words to ids.  In this function, create two dictionaries:
- Dictionary to go from the words to an id, we'll call `vocab_to_int`
- Dictionary to go from the id to word, we'll call `int_to_vocab`

Return these dictionaries in the following **tuple** `(vocab_to_int, int_to_vocab)`

In [3]:
import problem_unittests as tests

def create_lookup_tables(text):
    """
    Create lookup tables for vocabulary
    :param text: The text of tv scripts split into words
    :return: A tuple of dicts (vocab_to_int, int_to_vocab)
    """
    # TODO: Implement Function
    vocab_to_int = {word: i for i, word in enumerate(set(text))}
    int_to_vocab = {i: word for word, i in vocab_to_int.items()}
    # return tuple
    return (vocab_to_int, int_to_vocab)


"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
tests.test_create_lookup_tables(create_lookup_tables)

Tests Passed


### Tokenize Punctuation
We'll be splitting the script into a word array using spaces as delimiters.  However, punctuations like periods and exclamation marks can create multiple ids for the same word. For example, "bye" and "bye!" would generate two different word ids.

Implement the function `token_lookup` to return a dict that will be used to tokenize symbols like "!" into "||Exclamation_Mark||".  Create a dictionary for the following symbols where the symbol is the key and value is the token:
- Period ( **.** )
- Comma ( **,** )
- Quotation Mark ( **"** )
- Semicolon ( **;** )
- Exclamation mark ( **!** )
- Question mark ( **?** )
- Left Parentheses ( **(** )
- Right Parentheses ( **)** )
- Dash ( **-** )
- Return ( **\n** )

This dictionary will be used to tokenize the symbols and add the delimiter (space) around it.  This separates each symbols as its own word, making it easier for the neural network to predict the next word. Make sure you don't use a value that could be confused as a word; for example, instead of using the value "dash", try using something like "||dash||".

In [4]:
def token_lookup():
    """
    Generate a dict to turn punctuation into a token.
    :return: Tokenized dictionary where the key is the punctuation and the value is the token
    """
    
    return {
        '.': '||period||',
        ',': '||comma||',
        ';': '||semicollon||',
        '"': '||quotationmark||',
        '!': '||exclamationmark||',
        '?': '||questionmark||',
        '(': '||leftparentheses||',
        ')': '||rightparentheses||',
        '-': '||dash||',
        '\n': '||return||',
    }

"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
tests.test_tokenize(token_lookup)

Tests Passed


## Pre-process all the data and save it

Running the code cell below will pre-process all the data and save it to file. You're encouraged to lok at the code for `preprocess_and_save_data` in the `helpers.py` file to see what it's doing in detail, but you do not need to change this code.

In [5]:
"""
DON'T MODIFY ANYTHING IN THIS CELL
"""
# pre-process training data
helper.preprocess_and_save_data(data_dir, token_lookup, create_lookup_tables)

# Check Point
This is your first checkpoint. If you ever decide to come back to this notebook or have to restart the notebook, you can start from here. The preprocessed data has been saved to disk.

In [6]:
"""
DON'T MODIFY ANYTHING IN THIS CELL
"""
import helper
import problem_unittests as tests

int_text, vocab_to_int, int_to_vocab, token_dict = helper.load_preprocess()

## Build the Neural Network
In this section, you'll build the components necessary to build an RNN by implementing the RNN Module and forward and backpropagation functions.

### Check Access to GPU

In [7]:
"""
DON'T MODIFY ANYTHING IN THIS CELL
"""
import torch

# Check for a GPU
train_on_gpu = torch.cuda.is_available()
if not train_on_gpu:
    print('No GPU found. Please use a GPU to train your neural network.')

No GPU found. Please use a GPU to train your neural network.


## Input
Let's start with the preprocessed input data. We'll use [TensorDataset](http://pytorch.org/docs/master/data.html#torch.utils.data.TensorDataset) to provide a known format to our dataset; in combination with [DataLoader](http://pytorch.org/docs/master/data.html#torch.utils.data.DataLoader), it will handle batching, shuffling, and other dataset iteration functions.

You can create data with TensorDataset by passing in feature and target tensors. Then create a DataLoader as usual.
```
data = TensorDataset(feature_tensors, target_tensors)
data_loader = torch.utils.data.DataLoader(data, 
                                          batch_size=batch_size)
```

### Batching
Implement the `batch_data` function to batch `words` data into chunks of size `batch_size` using the `TensorDataset` and `DataLoader` classes.

>You can batch words using the DataLoader, but it will be up to you to create `feature_tensors` and `target_tensors` of the correct size and content for a given `sequence_length`.

For example, say we have these as input:
```
words = [1, 2, 3, 4, 5, 6, 7]
sequence_length = 4
```

Your first `feature_tensor` should contain the values:
```
[1, 2, 3, 4]
```
And the corresponding `target_tensor` should just be the next "word"/tokenized word value:
```
5
```
This should continue with the second `feature_tensor`, `target_tensor` being:
```
[2, 3, 4, 5]  # features
6             # target
```

In [8]:
from torch.utils.data import TensorDataset, DataLoader


def batch_data(words, sequence_length, batch_size):
    """
    Batch the neural network data using DataLoader
    :param words: The word ids of the TV scripts
    :param sequence_length: The sequence length of each batch
    :param batch_size: The size of each batch; the number of sequences in a batch
    :return: DataLoader with batched data
    """
    # TODO: Implement function
    
    features = []
    targets = []
    for idx in range(0, len(words)-sequence_length):
        feature_tensor = words[idx:(idx+sequence_length)]
        features.append(feature_tensor)
        
        target_tensor = words[idx+sequence_length]
        targets.append(target_tensor)
    
    dataset = TensorDataset(torch.tensor(features), torch.tensor(targets))
    loader = DataLoader(dataset, batch_size=batch_size, shuffle=True)
    return loader

# there is no test for this function, but you are encouraged to create
# print statements and tests of your own
data_loader = batch_data([1, 2, 3, 4, 5, 6, 7], 4, 2)
sample_x, sample_y = next(iter(data_loader))
print(sample_x)
print(sample_y)

tensor([[2, 3, 4, 5],
        [1, 2, 3, 4]])
tensor([6, 5])


### Test your dataloader 

You'll have to modify this code to test a batching function, but it should look fairly similar.

Below, we're generating some test text data and defining a dataloader using the function you defined, above. Then, we are getting some sample batch of inputs `sample_x` and targets `sample_y` from our dataloader.

Your code should return something like the following (likely in a different order, if you shuffled your data):

```
torch.Size([10, 5])
tensor([[ 28,  29,  30,  31,  32],
        [ 21,  22,  23,  24,  25],
        [ 17,  18,  19,  20,  21],
        [ 34,  35,  36,  37,  38],
        [ 11,  12,  13,  14,  15],
        [ 23,  24,  25,  26,  27],
        [  6,   7,   8,   9,  10],
        [ 38,  39,  40,  41,  42],
        [ 25,  26,  27,  28,  29],
        [  7,   8,   9,  10,  11]])

torch.Size([10])
tensor([ 33,  26,  22,  39,  16,  28,  11,  43,  30,  12])
```

### Sizes
Your sample_x should be of size `(batch_size, sequence_length)` or (10, 5) in this case and sample_y should just have one dimension: batch_size (10). 

### Values

You should also notice that the targets, sample_y, are the *next* value in the ordered test_text data. So, for an input sequence `[ 28,  29,  30,  31,  32]` that ends with the value `32`, the corresponding output should be `33`.

In [9]:
# test dataloader

test_text = range(50)
t_loader = batch_data(test_text, sequence_length=5, batch_size=10)

data_iter = iter(t_loader)
sample_x, sample_y = data_iter.next()

print(sample_x.shape)
print(sample_x)
print()
print(sample_y.shape)
print(sample_y)

torch.Size([10, 5])
tensor([[13, 14, 15, 16, 17],
        [37, 38, 39, 40, 41],
        [23, 24, 25, 26, 27],
        [ 2,  3,  4,  5,  6],
        [ 5,  6,  7,  8,  9],
        [ 9, 10, 11, 12, 13],
        [44, 45, 46, 47, 48],
        [18, 19, 20, 21, 22],
        [41, 42, 43, 44, 45],
        [42, 43, 44, 45, 46]])

torch.Size([10])
tensor([18, 42, 28,  7, 10, 14, 49, 23, 46, 47])


---
## Build the Neural Network
Implement an RNN using PyTorch's [Module class](http://pytorch.org/docs/master/nn.html#torch.nn.Module). You may choose to use a GRU or an LSTM. To complete the RNN, you'll have to implement the following functions for the class:
 - `__init__` - The initialize function. 
 - `init_hidden` - The initialization function for an LSTM/GRU hidden state
 - `forward` - Forward propagation function.
 
The initialize function should create the layers of the neural network and save them to the class. The forward propagation function will use these layers to run forward propagation and generate an output and a hidden state.

**The output of this model should be the *last* batch of word scores** after a complete sequence has been processed. That is, for each input sequence of words, we only want to output the word scores for a single, most likely, next word.

### Hints

1. Make sure to stack the outputs of the lstm to pass to your fully-connected layer, you can do this with `lstm_output = lstm_output.contiguous().view(-1, self.hidden_dim)`
2. You can get the last batch of word scores by shaping the output of the final, fully-connected layer like so:

```
# reshape into (batch_size, seq_length, output_size)
output = output.view(batch_size, -1, self.output_size)
# get last batch
out = output[:, -1]
```

In [10]:
import torch.nn as nn

In [11]:
import torch.nn as nn

class RNN(nn.Module):
    
    def __init__(self, vocab_size, output_size, embedding_dim, hidden_dim, n_layers, dropout=0.5):
        """
        Initialize the PyTorch RNN Module
        :param vocab_size: The number of input dimensions of the neural network (the size of the vocabulary)
        :param output_size: The number of output dimensions of the neural network
        :param embedding_dim: The size of embeddings, should you choose to use them        
        :param hidden_dim: The size of the hidden layer outputs
        :param dropout: dropout to add in between LSTM/GRU layers
        """
        super(RNN, self).__init__()
        # TODO: Implement function
        
        # set class variables
        self.dropout = dropout
        self.n_layers = n_layers
        self.hidden_dim = hidden_dim
        self.embedding_dim = embedding_dim
        self.output_size = output_size
        
        # define model layers
        self.embedding = nn.Embedding(num_embeddings=vocab_size, embedding_dim=embedding_dim)
        self.lstm = nn.LSTM(embedding_dim, hidden_size=hidden_dim, num_layers=n_layers, dropout=dropout, batch_first=True)
        self.fc = nn.Linear(in_features=hidden_dim, out_features=output_size)
    
    
    def forward(self, nn_input, hidden):
        """
        Forward propagation of the neural network
        :param nn_input: The input to the neural network
        :param hidden: The hidden state        
        :return: Two Tensors, the output of the neural network and the latest hidden state
        """
        batch_size = nn_input.size(0)
        nn_input = nn_input.long()
        embeds = self.embedding(nn_input)
        
        lstm_out, hidden = self.lstm(embeds, hidden)
        lstm_out = lstm_out.contiguous().reshape(-1, self.hidden_dim)
        
        out = self.fc(lstm_out)
        out = out.view(batch_size, -1, self.output_size)
        out = out[:, -1] # get last batch of labels
        
        # return one batch of output word scores and the hidden state
        return out, hidden
    
    
    def init_hidden(self, batch_size):
        '''
        Initialize the hidden state of an LSTM/GRU
        :param batch_size: The batch_size of the hidden state
        :return: hidden state of dims (n_layers, batch_size, hidden_dim)
        '''
        # Implement function
        
        # initialize hidden state with zero weights, and move to GPU if available
        
        weight = next(self.parameters()).data
        
        if (train_on_gpu):
            hidden = (weight.new(self.n_layers, batch_size, self.hidden_dim).zero_().cuda(),
                  weight.new(self.n_layers, batch_size, self.hidden_dim).zero_().cuda())
        else:
            hidden = (weight.new(self.n_layers, batch_size, self.hidden_dim).zero_(),
                      weight.new(self.n_layers, batch_size, self.hidden_dim).zero_())
        
        return hidden

"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
tests.test_rnn(RNN, train_on_gpu)

Tests Passed


### Define forward and backpropagation

Use the RNN class you implemented to apply forward and back propagation. This function will be called, iteratively, in the training loop as follows:
```
loss = forward_back_prop(decoder, decoder_optimizer, criterion, inp, target)
```

And it should return the average loss over a batch and the hidden state returned by a call to `RNN(inp, hidden)`. Recall that you can get this loss by computing it, as usual, and calling `loss.item()`.

**If a GPU is available, you should move your data to that GPU device, here.**

In [12]:
def forward_back_prop(rnn, optimizer, criterion, inp, target, hidden):
    """
    Forward and backward propagation on the neural network
    :param rnn: The PyTorch Module that holds the neural network
    :param optimizer: The PyTorch optimizer for the neural network
    :param criterion: The PyTorch loss function
    :param inp: A batch of input to the neural network
    :param target: The target output for the batch of input
    :return: The loss and the latest hidden state Tensor
    """
    
    # TODO: Implement Function
    
    # move data to GPU, if available
    if train_on_gpu:
        inp, target = inp.cuda(), target.cuda()
    
    hidden = tuple([each.data for each in hidden])
    
    # perform backpropagation and optimization
    rnn.zero_grad()
    
    out, hidden = rnn(inp, hidden)
    loss = criterion(out.squeeze(), target)
    loss.backward(retain_graph=True)
    nn.utils.clip_grad_norm_(rnn.parameters(), 5)
    
    optimizer.step()
    
    
    # return the loss over a batch and the hidden state produced by our model
    return loss.item(), hidden

# Note that these tests aren't completely extensive.
# they are here to act as general checks on the expected outputs of your functions
"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
tests.test_forward_back_prop(RNN, forward_back_prop, train_on_gpu)

Tests Passed


## Neural Network Training

With the structure of the network complete and data ready to be fed in the neural network, it's time to train it.

### Train Loop

The training loop is implemented for you in the `train_decoder` function. This function will train the network over all the batches for the number of epochs given. The model progress will be shown every number of batches. This number is set with the `show_every_n_batches` parameter. You'll set this parameter along with other parameters in the next section.

In [13]:
"""
DON'T MODIFY ANYTHING IN THIS CELL
"""

def train_rnn(rnn, batch_size, optimizer, criterion, n_epochs, show_every_n_batches=100):
    batch_losses = []
    
    rnn.train()

    print("Training for %d epoch(s)..." % n_epochs)
    for epoch_i in range(1, n_epochs + 1):
        
        # initialize hidden state
        hidden = rnn.init_hidden(batch_size)
        
        for batch_i, (inputs, labels) in enumerate(train_loader, 1):
            
#             print("inputs.shape: {}\nlabels.shape: {}".format(inputs.shape, labels.shape))
            # make sure you iterate over completely full batches, only
            n_batches = len(train_loader.dataset)//batch_size
            if(batch_i > n_batches):
                break
            
            # forward, back prop
            loss, hidden = forward_back_prop(rnn, optimizer, criterion, inputs, labels, hidden)          
            # record loss
            batch_losses.append(loss)

            # printing loss stats
            if batch_i % show_every_n_batches == 0:
                print('Epoch: {:>4}/{:<4}  Loss: {}\n'.format(
                    epoch_i, n_epochs, np.average(batch_losses)))
                batch_losses = []

    # returns a trained rnn
    return rnn

### Hyperparameters

Set and train the neural network with the following parameters:
- Set `sequence_length` to the length of a sequence.
- Set `batch_size` to the batch size.
- Set `num_epochs` to the number of epochs to train for.
- Set `learning_rate` to the learning rate for an Adam optimizer.
- Set `vocab_size` to the number of unique tokens in our vocabulary.
- Set `output_size` to the desired size of the output.
- Set `embedding_dim` to the embedding dimension; smaller than the vocab_size.
- Set `hidden_dim` to the hidden dimension of your RNN.
- Set `n_layers` to the number of layers/cells in your RNN.
- Set `show_every_n_batches` to the number of batches at which the neural network should print progress.

If the network isn't getting the desired results, tweak these parameters and/or the layers in the `RNN` class.

In [14]:
# Data params
# Sequence Length
sequence_length = 10  # of words in a sequence
# Batch Size
batch_size = 128

# data loader - do not change
train_loader = batch_data(int_text, sequence_length, batch_size)

In [15]:
# Training parameters
# Number of Epochs
num_epochs = 10
# Learning Rate
learning_rate = 0.001

# Model parameters
# Vocab size
vocab_size = len(vocab_to_int)
# Output size
output_size = len(vocab_to_int)
# Embedding Dimension
embedding_dim = 200
# Hidden Dimension
hidden_dim = 256
# Number of RNN Layers
n_layers = 2

# Show stats for every n number of batches
show_every_n_batches = 25

### Train
In the next cell, you'll train the neural network on the pre-processed data.  If you have a hard time getting a good loss, you may consider changing your hyperparameters. In general, you may get better results with larger hidden and n_layer dimensions, but larger models take a longer time to train. 
> **You should aim for a loss less than 3.5.** 

You should also experiment with different sequence lengths, which determine the size of the long range dependencies that a model can learn.

In [16]:
"""
DON'T MODIFY ANYTHING IN THIS CELL
"""

# create model and move to gpu if available
rnn = RNN(vocab_size, output_size, embedding_dim, hidden_dim, n_layers, dropout=0.5)
if train_on_gpu:
    rnn.cuda()

# defining loss and optimization functions for training
optimizer = torch.optim.Adam(rnn.parameters(), lr=learning_rate)
criterion = nn.CrossEntropyLoss()

# training the model
trained_rnn = train_rnn(rnn, batch_size, optimizer, criterion, num_epochs, show_every_n_batches)

# saving the trained model
helper.save_model('./save/trained_rnn', trained_rnn)
print('Model Trained and Saved')

Training for 10 epoch(s)...
Epoch:    1/10    Loss: 7.812271919250488

Epoch:    1/10    Loss: 6.17623010635376

Epoch:    1/10    Loss: 6.012040748596191

Epoch:    1/10    Loss: 5.936744441986084

Epoch:    1/10    Loss: 5.695862693786621

Epoch:    1/10    Loss: 5.560325736999512

Epoch:    1/10    Loss: 5.572571773529052

Epoch:    1/10    Loss: 5.510920257568359

Epoch:    1/10    Loss: 5.480544624328613

Epoch:    1/10    Loss: 5.284591274261475

Epoch:    1/10    Loss: 5.234848403930664

Epoch:    1/10    Loss: 5.3087772941589355

Epoch:    1/10    Loss: 5.274135532379151

Epoch:    1/10    Loss: 5.218908977508545

Epoch:    1/10    Loss: 5.17513557434082

Epoch:    1/10    Loss: 5.201657829284668

Epoch:    1/10    Loss: 5.183044300079346

Epoch:    1/10    Loss: 5.135552082061768

Epoch:    1/10    Loss: 5.178689479827881

Epoch:    1/10    Loss: 5.082576389312744

Epoch:    1/10    Loss: 5.023716030120849

Epoch:    1/10    Loss: 5.11957166671753

Epoch:    1/10    Loss: 5.05

Epoch:    1/10    Loss: 4.198865070343017

Epoch:    1/10    Loss: 4.170787363052368

Epoch:    1/10    Loss: 4.311856336593628

Epoch:    1/10    Loss: 4.234275312423706

Epoch:    1/10    Loss: 4.18063889503479

Epoch:    1/10    Loss: 4.2252506828308105

Epoch:    1/10    Loss: 4.246142559051513

Epoch:    1/10    Loss: 4.1829811954498295

Epoch:    1/10    Loss: 4.199199333190918

Epoch:    1/10    Loss: 4.249093427658081

Epoch:    1/10    Loss: 4.341059064865112

Epoch:    1/10    Loss: 4.2108802318573

Epoch:    1/10    Loss: 4.268321418762207

Epoch:    1/10    Loss: 4.215850915908813

Epoch:    1/10    Loss: 4.162393236160279

Epoch:    1/10    Loss: 4.119665756225586

Epoch:    1/10    Loss: 4.158965034484863

Epoch:    1/10    Loss: 4.223981065750122

Epoch:    1/10    Loss: 4.353611803054809

Epoch:    1/10    Loss: 4.204665012359619

Epoch:    1/10    Loss: 4.105412826538086

Epoch:    1/10    Loss: 4.159495010375976

Epoch:    1/10    Loss: 4.156162729263306

Epoch:    1/

Epoch:    2/10    Loss: 3.9277514266967772

Epoch:    2/10    Loss: 3.944080629348755

Epoch:    2/10    Loss: 3.9453007221221923

Epoch:    2/10    Loss: 4.025981054306031

Epoch:    2/10    Loss: 3.9980738735198975

Epoch:    2/10    Loss: 4.000446786880493

Epoch:    2/10    Loss: 3.962170295715332

Epoch:    2/10    Loss: 3.8634063243865966

Epoch:    2/10    Loss: 4.003384332656861

Epoch:    2/10    Loss: 3.9120828437805177

Epoch:    2/10    Loss: 3.9418318176269533

Epoch:    2/10    Loss: 3.905043535232544

Epoch:    2/10    Loss: 3.9492342281341553

Epoch:    2/10    Loss: 3.9449239063262937

Epoch:    2/10    Loss: 4.085811386108398

Epoch:    2/10    Loss: 3.9256765842437744

Epoch:    2/10    Loss: 4.0397271728515625

Epoch:    2/10    Loss: 3.8992686080932617

Epoch:    2/10    Loss: 4.02645884513855

Epoch:    2/10    Loss: 3.911669511795044

Epoch:    2/10    Loss: 3.9468257999420167

Epoch:    2/10    Loss: 3.904152011871338

Epoch:    2/10    Loss: 3.951222848892212



Epoch:    3/10    Loss: 3.7379517936706543

Epoch:    3/10    Loss: 3.7970232009887694

Epoch:    3/10    Loss: 3.7251721954345705

Epoch:    3/10    Loss: 3.8814155292510986

Epoch:    3/10    Loss: 3.8462788105010985

Epoch:    3/10    Loss: 3.8008359622955323

Epoch:    3/10    Loss: 3.7843207836151125

Epoch:    3/10    Loss: 3.7141612434387206

Epoch:    3/10    Loss: 3.795104913711548

Epoch:    3/10    Loss: 3.705536813735962

Epoch:    3/10    Loss: 3.77871205329895

Epoch:    3/10    Loss: 3.8074650382995605

Epoch:    3/10    Loss: 3.830361557006836

Epoch:    3/10    Loss: 3.766918125152588

Epoch:    3/10    Loss: 3.725276756286621

Epoch:    3/10    Loss: 3.730824575424194

Epoch:    3/10    Loss: 3.7749496269226075

Epoch:    3/10    Loss: 3.7540923023223876

Epoch:    3/10    Loss: 3.8084644985198977

Epoch:    3/10    Loss: 3.8774134063720704

Epoch:    3/10    Loss: 3.7685462474822997

Epoch:    3/10    Loss: 3.7411092472076417

Epoch:    3/10    Loss: 3.78511283874511

Epoch:    3/10    Loss: 3.737086935043335

Epoch:    3/10    Loss: 3.7620388889312744

Epoch:    3/10    Loss: 3.704517517089844

Epoch:    3/10    Loss: 3.8075361156463625

Epoch:    3/10    Loss: 3.826813163757324

Epoch:    3/10    Loss: 3.7929573535919188

Epoch:    3/10    Loss: 3.7311591720581054

Epoch:    3/10    Loss: 3.8007006549835207

Epoch:    3/10    Loss: 3.768501672744751

Epoch:    3/10    Loss: 3.687505292892456

Epoch:    3/10    Loss: 3.6961089324951173

Epoch:    3/10    Loss: 3.747580299377441

Epoch:    3/10    Loss: 3.8393594360351564

Epoch:    3/10    Loss: 3.830522365570068

Epoch:    3/10    Loss: 3.7926807498931883

Epoch:    3/10    Loss: 3.8476698207855224

Epoch:    3/10    Loss: 3.7817974281311035

Epoch:    3/10    Loss: 3.6941195583343505

Epoch:    3/10    Loss: 3.6741196727752685

Epoch:    3/10    Loss: 3.737931041717529

Epoch:    3/10    Loss: 3.7196802997589113

Epoch:    3/10    Loss: 3.8532392311096193

Epoch:    3/10    Loss: 3.69846692085266

...

Epoch:    8/10    Loss: 3.294211196899414

Epoch:    8/10    Loss: 3.42994740486145

Epoch:    8/10    Loss: 3.4127158737182617

Epoch:    8/10    Loss: 3.3985971927642824

Epoch:    8/10    Loss: 3.294696922302246

Epoch:    8/10    Loss: 3.389333362579346

Epoch:    8/10    Loss: 3.361317720413208

Epoch:    8/10    Loss: 3.4500469875335695

Epoch:    8/10    Loss: 3.3850589275360106

Epoch:    8/10    Loss: 3.4114867877960204

Epoch:    8/10    Loss: 3.361394834518433

Epoch:    8/10    Loss: 3.4069757652282715

Epoch:    8/10    Loss: 3.375915832519531

Epoch:    8/10    Loss: 3.4514839363098146

Epoch:    8/10    Loss: 3.414586753845215

Epoch:    8/10    Loss: 3.414146327972412

Epoch:    8/10    Loss: 3.405916805267334

Epoch:    8/10    Loss: 3.3954063510894774

Epoch:    8/10    Loss: 3.322054376602173

Epoch:    8/10    Loss: 3.440255880355835

Epoch:    8/10    Loss: 3.384837532043457

Epoch:    8/10    Loss: 3.436966648101807

Epoch:    8/10    Loss: 3.4471005249023436

Epo

Epoch:    9/10    Loss: 3.3315474605560302

Epoch:    9/10    Loss: 3.3257224655151365

Epoch:    9/10    Loss: 3.3462365436553956

Epoch:    9/10    Loss: 3.317899236679077

Epoch:    9/10    Loss: 3.3466105365753176

Epoch:    9/10    Loss: 3.340748233795166

Epoch:    9/10    Loss: 3.2876572895050047

Epoch:    9/10    Loss: 3.2338710689544676

Epoch:    9/10    Loss: 3.230593566894531

Epoch:    9/10    Loss: 3.365699596405029

Epoch:    9/10    Loss: 3.336807622909546

Epoch:    9/10    Loss: 3.251954154968262

Epoch:    9/10    Loss: 3.3284068870544434

Epoch:    9/10    Loss: 3.414784231185913

Epoch:    9/10    Loss: 3.3177080059051516

Epoch:    9/10    Loss: 3.337831335067749

Epoch:    9/10    Loss: 3.2941868019104006

Epoch:    9/10    Loss: 3.3557864475250243

Epoch:    9/10    Loss: 3.283981485366821

Epoch:    9/10    Loss: 3.2869751834869385

Epoch:    9/10    Loss: 3.3396662139892577

Epoch:    9/10    Loss: 3.405130968093872

Epoch:    9/10    Loss: 3.331829710006714


Epoch:    9/10    Loss: 3.436304693222046

Epoch:    9/10    Loss: 3.386737823486328

Epoch:    9/10    Loss: 3.4189165115356444

Epoch:    9/10    Loss: 3.4846177387237547

Epoch:    9/10    Loss: 3.3741824054718017

Epoch:    9/10    Loss: 3.3832020282745363

Epoch:    9/10    Loss: 3.4803872299194336

Epoch:    9/10    Loss: 3.4127313899993896

Epoch:    9/10    Loss: 3.392734546661377

Epoch:    9/10    Loss: 3.4114136695861816

Epoch:    9/10    Loss: 3.4327598285675047

Epoch:    9/10    Loss: 3.3952191257476807

Epoch:    9/10    Loss: 3.4529370212554933

Epoch:    9/10    Loss: 3.4268124961853026

Epoch:    9/10    Loss: 3.4001009273529053

Epoch:    9/10    Loss: 3.3425535202026366

Epoch:    9/10    Loss: 3.45607084274292

Epoch:    9/10    Loss: 3.4760212802886965

Epoch:    9/10    Loss: 3.343898458480835

Epoch:    9/10    Loss: 3.4802566814422606

Epoch:    9/10    Loss: 3.4557285499572754

Epoch:    9/10    Loss: 3.3529163646697997

Epoch:    9/10    Loss: 3.425932264328

Epoch:   10/10    Loss: 3.247282886505127

Epoch:   10/10    Loss: 3.318057918548584

Epoch:   10/10    Loss: 3.3050889682769777

Epoch:   10/10    Loss: 3.2979873180389405

Epoch:   10/10    Loss: 3.3058952140808104

Epoch:   10/10    Loss: 3.3291269016265868

Epoch:   10/10    Loss: 3.365081958770752

Epoch:   10/10    Loss: 3.3828722476959228

Epoch:   10/10    Loss: 3.398787088394165

Epoch:   10/10    Loss: 3.260361270904541

Epoch:   10/10    Loss: 3.3173880481719973

Epoch:   10/10    Loss: 3.3683952713012695

Epoch:   10/10    Loss: 3.3799635982513427

Epoch:   10/10    Loss: 3.3868644618988037

Epoch:   10/10    Loss: 3.32786171913147

Epoch:   10/10    Loss: 3.2738842010498046

Epoch:   10/10    Loss: 3.290056610107422

Epoch:   10/10    Loss: 3.2680533599853514

Epoch:   10/10    Loss: 3.321265153884888

Epoch:   10/10    Loss: 3.3418817043304445

Epoch:   10/10    Loss: 3.4192684364318846

Epoch:   10/10    Loss: 3.3741913414001465

Epoch:   10/10    Loss: 3.291375970840454

/miniconda3/envs/deep-learning/lib/python3.6/site-packages/torch/serialization.py:251: UserWarning: Couldn't retrieve source code for container of type RNN. It won't be checked for correctness upon loading.
  "type " + obj.__name__ + ". It won't be checked "


### Question: How did you decide on your model hyperparameters? 
For example, did you try different sequence_lengths and find that one size made the model converge faster? What about your hidden_dim and n_layers; how did you decide on those?

**Answer:** I mostly varied `sequence_length` and `hidden_dim` and watched how the first batches evolved. A big factor was the memory usage, so I kept `sequence_length` in a low value, as higer values would simply crash my computer. With `n_layers` I went with the recommended 2 layers, and with batch size I went with 128, a power of 2, and also a recurrent value in literature.

---
# Checkpoint

After running the above training cell, your model will be saved by name, `trained_rnn`, and if you save your notebook progress, **you can pause here and come back to this code at another time**. You can resume your progress by running the next cell, which will load in our word:id dictionaries _and_ load in your saved model by name!

In [21]:
"""
DON'T MODIFY ANYTHING IN THIS CELL
"""
import torch
import helper
import problem_unittests as tests

_, vocab_to_int, int_to_vocab, token_dict = helper.load_preprocess()
trained_rnn = helper.load_model('./save/trained_rnn')

## Generate TV Script
With the network trained and saved, you'll use it to generate a new, "fake" Seinfeld TV script in this section.

### Generate Text
To generate the text, the network needs to start with a single word and repeat its predictions until it reaches a set length. You'll be using the `generate` function to do this. It takes a word id to start with, `prime_id`, and generates a set length of text, `predict_len`. Also note that it uses topk sampling to introduce some randomness in choosing the most likely next word, given an output set of word scores!

In [22]:
"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
import torch.nn.functional as F

def generate(rnn, prime_id, int_to_vocab, token_dict, pad_value, predict_len=100):
    """
    Generate text using the neural network
    :param decoder: The PyTorch Module that holds the trained neural network
    :param prime_id: The word id to start the first prediction
    :param int_to_vocab: Dict of word id keys to word values
    :param token_dict: Dict of puncuation tokens keys to puncuation values
    :param pad_value: The value used to pad a sequence
    :param predict_len: The length of text to generate
    :return: The generated text
    """
    rnn.eval()
    
    # create a sequence (batch_size=1) with the prime_id
    current_seq = np.full((1, sequence_length), pad_value)
    current_seq[-1][-1] = prime_id
    predicted = [int_to_vocab[prime_id]]
    
    for _ in range(predict_len):
        if train_on_gpu:
            current_seq = torch.LongTensor(current_seq).cuda()
        else:
            current_seq = torch.LongTensor(current_seq)
        
        # initialize the hidden state
        hidden = rnn.init_hidden(current_seq.size(0))
        
        # get the output of the rnn
        output, _ = rnn(current_seq, hidden)
        
        # get the next word probabilities
        p = F.softmax(output, dim=1).data
        if(train_on_gpu):
            p = p.cpu() # move to cpu
         
        # use top_k sampling to get the index of the next word
        top_k = 5
        p, top_i = p.topk(top_k)
        top_i = top_i.numpy().squeeze()
        
        # select the likely next word index with some element of randomness
        p = p.numpy().squeeze()
        word_i = np.random.choice(top_i, p=p/p.sum())
        
        # retrieve that word from the dictionary
        word = int_to_vocab[word_i]
        predicted.append(word)     
        
        # the generated word becomes the next "current sequence" and the cycle can continue
        current_seq = np.roll(current_seq, -1, 1)
        current_seq[-1][-1] = word_i
    
    gen_sentences = ' '.join(predicted)
    
    # Replace punctuation tokens
    for key, token in token_dict.items():
        ending = ' ' if key in ['\n', '(', '"'] else ''
        gen_sentences = gen_sentences.replace(' ' + token.lower(), key)
    gen_sentences = gen_sentences.replace('\n ', '\n')
    gen_sentences = gen_sentences.replace('( ', '(')
    
    # return all the sentences
    return gen_sentences

### Generate a New Script
It's time to generate the text. Set `gen_length` to the length of TV script you want to generate and set `prime_word` to one of the following to start the prediction:
- "jerry"
- "elaine"
- "george"
- "kramer"

You can set the prime word to _any word_ in our dictionary, but it's best to start with a name for generating a TV script. (You can also start with any other names you find in the original text file!)

In [23]:
# run the cell multiple times to get different results!
gen_length = 400 # modify the length to your preference
prime_word = 'jerry' # name for starting the script

"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
pad_word = helper.SPECIAL_WORDS['PADDING']
generated_script = generate(trained_rnn, vocab_to_int[prime_word + ':'], int_to_vocab, token_dict, vocab_to_int[pad_word], gen_length)
print(generated_script)

jerry: oy- deon- ba- boom- la- la, hoo, hee- e!(laughs)

[setting: jerry's office]

george:(still laughing) i think i could take a little more.

george: what do you want me to say?

jerry: i didn't know what i mean, i think he was gonna be a big trouble, but i can't.

george: you know, i'm sure that we have the idea.

kramer:(to the phone) oh, i can't get it.

elaine: well, what is it about?

jerry: i know.

jerry: i didn't know that.

newman: i don't think so...

jerry:(on intercom) oh.

george: hey, how are you gonna be?! i can't believe it was a great one thing for me.

jerry: well, i'm sure you could be a little nervous.

jerry: what?

george: well, i think we should go.

elaine: i don't even know.

kramer: well.....

george: i don't know.... you know, i'm gonna go out to a doctor.

george: what?

jerry: well, what do you think about it?

kramer: i don't know how much i need it.

george: i can't believe this.(jerry shakes his head 'no'. jerry and george are looking up at the airpor

#### Save your favorite scripts

Once you have a script that you like (or find interesting), save it to a text file!

In [24]:
# save script to a text file
f =  open("generated_script_1.txt","w")
f.write(generated_script)
f.close()

# The TV Script is Not Perfect
It's ok if the TV script doesn't make perfect sense. It should look like alternating lines of dialogue, here is one such example of a few generated lines.

### Example generated script

>jerry: what about me?
>
>jerry: i don't have to wait.
>
>kramer:(to the sales table)
>
>elaine:(to jerry) hey, look at this, i'm a good doctor.
>
>newman:(to elaine) you think i have no idea of this...
>
>elaine: oh, you better take the phone, and he was a little nervous.
>
>kramer:(to the phone) hey, hey, jerry, i don't want to be a little bit.(to kramer and jerry) you can't.
>
>jerry: oh, yeah. i don't even know, i know.
>
>jerry:(to the phone) oh, i know.
>
>kramer:(laughing) you know...(to jerry) you don't know.

You can see that there are multiple characters that say (somewhat) complete sentences, but it doesn't have to be perfect! It takes quite a while to get good results, and often, you'll have to use a smaller vocabulary (and discard uncommon words), or get more data.  The Seinfeld dataset is about 3.4 MB, which is big enough for our purposes; for script generation you'll want more than 1 MB of text, generally. 

# Submitting This Project
When submitting this project, make sure to run all the cells before saving the notebook. Save the notebook file as "dlnd_tv_script_generation.ipynb" and save another copy as an HTML file by clicking "File" -> "Download as.."->"html". Include the "helper.py" and "problem_unittests.py" files in your submission. Once you download these files, compress them into one zip file for submission.